In [ ]:
import pandas as pd
import datetime
import numpy as np
import seaborn as sns; sns.set()
import matplotlib
import matplotlib.pyplot as plt

%matplotlib auto

In [ ]:
demand_df = pd.read_csv(r'hourly_demand_all_regions.csv', encoding='latin-1')
demand_df = pd.DataFrame(demand_df)
demand_df.rename({'Año':'Year', 'Mes':'Month', 'Día':'Day', 'Hora':'Hour'}, inplace=True, axis=1)
demand_df.drop(['La Paz', 'SIN', 'SIN8', 'SIN8+LPZ'], axis=1, inplace=True)
demand_df['09-Baja California Sur'] = demand_df['09-Baja California Sur'] - demand_df['10-Mulegé']
#demand_df = pd.melt(demand_df, id_vars= ['Year', 'Month','Day','Hour'], value_vars=[x for x in demand_df.columns if x not in ['Year', 'Month','Day','Hour']])
demand_df.rename({'variable':'Region'}, inplace=True, axis=1)
#demand_df.drop(['Day','Hour'], axis=1, inplace=True)

# Create column for weekday/weekend
demand_df['Date'] = demand_df['Year'].astype(str) + '-' + demand_df['Month'].astype(str) + '-' + demand_df['Day'].astype(str)
demand_df['Date'] = pd.to_datetime(demand_df['Date'], errors='coerce', format='%Y-%m-%d')
demand_df['Day-of-week'] = demand_df['Date'].dt.dayofweek
demand_df.drop('Date', axis=1, inplace=True)
demand_df['Day-of-week'][demand_df['Day-of-week']<5] = 'WD'
demand_df['Day-of-week'][demand_df['Day-of-week']!='WD'] = 'WE'


print(demand_df['Day-of-week'].unique())
print(demand_df['Year'].unique())

In [ ]:
seasons = {
    1:'SEA1',
    2:'SEA1',
    3:'SEA2',
    4:'SEA2',
    5:'SEA2',
    6:'SEA3',
    7:'SEA3',
    8:'SEA3',
    9:'SEA4',
    10:'SEA4',
    11:'SEA4',
    12:'SEA1',
}

day_parts = {
    'DP1':[0,7],
    'DP2':[7,14],
    'DP3':[14,21],
    'DP4':[21,24],
    
}

timeslices = {}


regions = [x for x in demand_df.columns if x not in ['Hour', 'Day', 'Month', 'Year', 'Day-of-week']]
years = demand_df['Year'].unique()


demand_df_hours = pd.DataFrame(columns=demand_df.columns)

for each_year in years:
    demand_df_year = demand_df[demand_df['Year']==each_year]
    demand_df_year.insert(4,'Hour_continuous',range(1,1+len(demand_df_year)))
    demand_df_hours = demand_df_hours.append(demand_df_year)

demand_df_hours['Month'].replace(seasons, inplace=True)
demand_df_hours['Total']  = demand_df_hours[regions].sum(axis=1)

df_yearsplit = demand_df_hours[['Total', 'Month', 'Day-of-week', 'Hour', 'Year']]
df_yearsplit = df_yearsplit[df_yearsplit['Year']==2018]

for each_season in df_yearsplit['Month'].unique():
    for each_day in df_yearsplit['Day-of-week'].unique():
        df_temp = df_yearsplit[(df_yearsplit['Month']==each_season) 
                           & (df_yearsplit['Day-of-week'] == each_day)].sort_values(by='Total')
        df_temp = df_temp.groupby(['Month', 'Day-of-week', 'Hour', 'Year']).mean().sort_values(by='Total')
        
        for each_DP in day_parts:
            #print(each_DP, day_parts[each_DP][0], day_parts[each_DP][1])
            df_temp_2 = df_temp[day_parts[each_DP][0]:day_parts[each_DP][1]]
            df_temp_2['Day_part'] = each_DP
            df_temp_2 = df_temp_2.reset_index()
            df_temp_2['Timeslice_key'] = df_temp_2['Month'] + df_temp_2['Day-of-week'] + df_temp_2['Hour'].astype(str)
            df_temp_2['Timeslice_value'] = df_temp_2['Month'] + df_temp_2['Day-of-week'] + df_temp_2['Day_part']
            #print(df_temp_2)
            timeslice_dict = dict(zip(df_temp_2['Timeslice_key'], 
                                      df_temp_2['Timeslice_value']))
            timeslices.update(timeslice_dict)

demand_df_hours['Timeslice'] = demand_df_hours['Month'] + demand_df_hours['Day-of-week'] + demand_df_hours['Hour'].astype(str)
#print(demand_df_hours)

demand_df_hours['Timeslice'].replace(timeslices, inplace=True)
print(demand_df_hours)


#demand_df = demand_df.groupby('Year').sum().mul(3.6*1e-6)

In [ ]:
## YearSplit calculation

demand_df_yearsplit = demand_df_hours.drop(['Total', 'Month', 'Day', 'Day-of-week', 'Hour', 'Hour_continuous'] 
                                           +regions, 
                                           axis=1)
demand_df_yearsplit = demand_df_yearsplit.groupby('Year')['Timeslice'].value_counts(normalize=True).to_frame('value')
demand_df_yearsplit = demand_df_yearsplit.reset_index()
demand_df_yearsplit = pd.pivot_table(demand_df_yearsplit, 
                                   index='Timeslice', 
                                   values='value', 
                                   columns='Year', 
                                   aggfunc=np.sum)

#print(demand_df_yearsplit)

In [ ]:
## SpecifiedDemandProfile calculation

demand_df_regions = demand_df_hours.drop(['Month', 'Day', 'Day-of-week', 'Hour', 'Hour_continuous'], axis=1)

demand_df_regions = demand_df_regions.groupby(['Year', 'Timeslice']).sum()
demand_df_total = demand_df_regions.groupby(['Year']).sum()

demand_df_regions[regions] = demand_df_regions[regions]/demand_df_total[regions]
demand_df_regions = demand_df_regions.reset_index()

demand_df_regions = pd.melt(demand_df_regions, 
                            id_vars=['Year', 'Timeslice'], 
                            value_vars=regions)


demand_df_regions = pd.pivot_table(demand_df_regions, 
                                   index=['variable','Timeslice'], 
                                   values='value', 
                                   columns='Year', 
                                   aggfunc=np.sum)

demand_df_total = demand_df_total.mul(3.6*1e-6)
demand_df_total = demand_df_total.reset_index()
demand_df_total = pd.melt(demand_df_total, 
                            id_vars='Year', 
                            value_vars=regions)
demand_df_total = pd.pivot_table(demand_df_total, 
                                   index='variable', 
                                   values='value', 
                                   columns='Year', 
                                   aggfunc=np.sum)
#print(demand_df_regions)

In [ ]:
demand_df_yearsplit.to_csv(r'yearsplit.csv')
demand_df_regions.to_csv(r'specified_demand_profiles.csv')
demand_df_total.to_csv(r'total_demand_PJ.csv')

In [ ]:
demand_df_pvcf = demand_df_hours.drop(regions +
                                     ['Month', 'Day', 'Hour', 'Day-of-week', 'Total'],
                                     axis=1)
demand_df_pvcf = demand_df_pvcf[demand_df_pvcf['Year']==2018].drop('Year', axis=1)

df_pvcf = pd.read_csv(r'hourly_pvdata.csv')

demand_df_pvcf = pd.merge(demand_df_pvcf, df_pvcf, on=['Hour_continuous']).drop('Hour_continuous', axis=1)
demand_df_pvcf = demand_df_pvcf.groupby(['Timeslice']).mean()
print(demand_df_pvcf)

demand_df_pvcf.to_csv(r'pv_cf_timeslices.csv')